In [2]:
pip install selenium

     |████████████████████████████████| 981 kB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 358 kB 52.6 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 23.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install mysql-connector-python-rf

     |████████████████████████████████| 11.9 MB 3.9 MB/s eta 0:00:01
  Created wheel for mysql-connector-python-rf: filename=mysql_connector_python_rf-2.2.2-cp39-cp39-macosx_10_9_x86_64.whl size=249480 sha256=e7a16ce71ccc70509cec735fcc0a0893d6f290048977dab03feaa70243e3cc4b
  Stored in directory: /Users/huiqiongwu/Library/Caches/pip/wheels/64/06/5b/ef9543936a3f5de15b02775b6ae548a09f3ed2319d20342771
Successfully built mysql-connector-python-rf
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
from selenium import webdriver
import json
import os
import pandas as pd
import numpy as np
import mysql.connector
from selenium.webdriver.common.keys import Keys

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36',
          'Accept-Language': 'en-US, en;q-0.5'
          }

In [13]:
URL = 'https://www.amazon.com/s?k=smart+tv'
prime_url = ''
non_prime_url = ''

for i in range(1, 10):
    ## navigate to the ith page
    responsei = requests.get(URL + "&page=" + str(i), headers = headers)
    soupi = BeautifulSoup(responsei.content, 'html.parser')
    
    ## Find the part of code containing prime information and the URL of each item
    link = soupi.find_all('div', {'class': 's-result-item', 'data-component-type': 's-search-result'})
   
    for j in range(0, len(link)):
        ## Find the sponsored items
        item = link[j].find_all('i', class_ = "a-icon")
        ## Get the URL of each item
        item_url = link[j].h2.a['href']
        ## Separate sponsored and non-sponsored items
        if len(item) == 0:
            non_prime_url = non_prime_url + '\n' + item_url
        else:
            prime_url = prime_url + '\n' + item_url
    
    ## Set a 2-second pause before going to the next page
    time.sleep(2)

In [14]:
## Save the URLs
with open('prime.txt', 'w') as file:
    file.write(prime_url.lstrip())
with open('non_prime.txt', 'w') as file:
    file.write(non_prime_url.lstrip())

In [5]:
## Create two folders
path1 = 'prime'
os.mkdir(path1)
path2 = 'non-prime'
os.mkdir(path2)

In [15]:
## Open the files
read_prime = open('prime.txt', 'r').read()
read_non_prime = open('non_prime.txt', 'r').read()
## Split the strings
prime_url_split = read_prime.split('\n')
non_prime_url_split = read_non_prime.split('\n')

In [24]:
## Download each prime item's URL into folder 'prime'
for item in range(0, len(prime_url_split)):
    URL = prime_url_split[item]
    response = requests.get('https://www.amazon.com' + URL, headers = headers)
    iid = str(item)
    
    with open(os.path.join(os.getcwd() + '/prime/' + iid + '.html'), 'w', encoding='utf8') as file:
        file.write(response.text)
    
    ## Set a 2-second pause before going to the next item
    time.sleep(2)

In [27]:
for item in range(0, len(non_prime_url_split)):
    URL = non_prime_url_split[item]
    response = requests.get('https://www.amazon.com' + URL, headers = headers)
    iid = str(item)
    
    with open(os.path.join(os.getcwd() + '/non-prime/' + iid + '.html'), 'w', encoding='utf8') as file:
        file.write(response.text)
    
    ## Set a 2-second pause before going to the next item
    time.sleep(2)

In [ ]:
## Loop through prime pages
pages = os.listdir('./prime')
prime_df = pd.DataFrame(columns = ['seller_name', 'seller_score', 'item_price', 'list_price','num_reviews', 'title'])
i = 0

for page in pages:
    readfile = open(os.path.join(os.getcwd() + '/prime', page), 'r',encoding='utf-8').read()
    soup = BeautifulSoup(readfile, 'html.parser')
    
    ## identify prime or non-prime item
    
    ## seller name
    try:
        find_seller = soup.find('div', id='bylineInfo_feature_div') ## selector command
        seller_name = find_seller.find('a', id='bylineInfo').get_text() ## selector command
    except:
        seller_name = None
    
    ## seller score
    try:
        seller_score = pd.to_numeric(find_seller.find('span', class_ = 'a-icon-alt').get_text()) ## selector command
    except:
        seller_score = None
    
    ## item price
    try:
        try:
            find_price = soup.find('span', class_='a-price aok-align-cnter reinventPricePriceToPayMargin pricetToPay').get_text() ## selector command
            item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price))

        except:
            find_price = soup.find('span', class_='a-price a-text-price a-size-medium apexPriceToPay a-offscreen').get_text() ## selector command
            item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price)) 
           
    except:
        item_price = None
    
    ## list price

    try:
        find_list_price = soup.find('div', class_='a-section a-spacing-small aok-align-center') ## selector command
        price_list = find_list_price.find('span', class_='a-offscreen').get_text()
        list_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', price_list))

    except:
        list_price = None
    
    ## number of reviews
    try:
        find_item = soup.find('div', id = 'centerlCol') ## selector command
        reviews = find_item.find('span', id = 'acrCustomerReviewText').get_text() ## selector command
        reviews_num = re.sub(r'([,\d]+) sold', r'\1', reviews)
        num_reviews = pd.to_numeric(re.sub(r',', r'', reviews_num))
    except:
        num_reviews = None
    
    ## title
    try:
        title_text = soup.find('h1', id = 'title').get_text() ## selector command
        title = re.sub(r'Details about  \xa0', r'', title_text)
    except:
        title = None
    
        
    ## condition
    try:
        condition = soup.find('div', id = 'vi-itm-cond').get_text() ## selector command
    except:
        condition = None
    
    ## Save to data frame
    prime_df.loc[i] = [seller_name, seller_score, item_price, list_price,num_reviews, title]
    i = i + 1

In [ ]:
## Loop through non-prime pages
pages = os.listdir('./non_prime')
nonprime_df = pd.DataFrame(columns = ['seller_name', 'seller_score', 'item_price', 'list price','num_reviews', 'title'])
i = 0

for page in pages:
    readfile = open(os.path.join(os.getcwd() + '/non_prime', page), 'r').read()
    soup = BeautifulSoup(readfile, 'html.parser')
    
    ## identify prime or non-prime item
    
    ## seller name
    try:
        find_seller = soup.find('div', id='bylineInfo_feature_div') ## selector command
        seller_name = find_seller.find('a', id='bylineInfo').get_text() ## selector command
    except:
        seller_name = None
    
    ## seller score
    try:
        seller_score = pd.to_numeric(find_seller.find('span', class_ = 'a-icon-alt').get_text()) ## selector command
    except:
        seller_score = None
    
    ## item price
    try:
        try:
            find_price = soup.find('span', class_='a-price aok-align-cnter reinventPricePriceToPayMargin pricetToPay').get_text() ## selector command
            item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price))

        except:
            find_price = soup.find('span', class_='a-price a-text-price a-size-medium apexPriceToPay a-offscreen').get_text() ## selector command
            item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price)) 
           
    except:
        item_price = None
    
    ## list price

    try:
        find_list_price = soup.find('div', class_='a-section a-spacing-small aok-align-center') ## selector command
        price_list = find_list_price.find('span', class_='a-offscreen').get_text()
        list_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', price_list))

    except:
        list_price = None
    
    ## number of reviews
    try:
        find_item = soup.find('div', id = 'centerlCol') ## selector command
        reviews = find_item.find('span', id = 'acrCustomerReviewText').get_text() ## selector command
        reviews_num = re.sub(r'([,\d]+) sold', r'\1', reviews)
        num_reviews = pd.to_numeric(re.sub(r',', r'', reviews_num))
    except:
        num_reviews = None
    
    ## title
    try:
        title_text = soup.find('h1', id = 'title').get_text() ## selector command
        title = re.sub(r'Details about  \xa0', r'', title_text)
    except:
        title = None
    
        
    ## condition
    try:
        condition = soup.find('div', id = 'vi-itm-cond').get_text() ## selector command
    except:
        condition = None
    
    ## Save to data frame
    nonprime_df.loc[i] = [seller_name, seller_score, item_price, list_price,num_reviews, title]
    i = i + 1